# Create zonal statistics and point extractions for comparing CONUS404 and reference datasets

<img src='./Eval_PreProc.svg' width=600>

The pre-processing step is needed in order to align the two datasets for analysis.  The specific 
steps needed to prepare a given dataset may differ, depending on the source and the variable of
interest. 

Some steps might include: 

* Organizing the time-series index such that the time steps for both simulated and observed are congruent
    * This may involve interpolation to estimate a more granular time-step than is found in the source data
    * More often, an agregating function is used to 'down-sample' the dataset to a coarser time step (days vs hours).
* Coordinate aggregation units between simulated and observed 
    * Gridded data may be sampled per HUC-12, HUC-6, etc. to match modeled data indexed by these units. 
    * Index formats may be adjusted (e.g. a 'gage_id' may be 'USGS-01104200' in one data set, vs '01104200' in another)
* Re-Chunking the data to make time-series analysis more efficient (see [here](/dev/null) for a primer on re-chunking).
<details>
  <summary>Guide to pre-requisites and learning outcomes...&lt;click to expand&gt;</summary>
  
  <table>
    <tr>
      <td>Pre-Requisites
      <td>To get the most out of this notebook, you should already have an understanding of these topics: 
        <ul>
        <li>pre-req one
        <li>pre-req two
        </ul>
    <tr>
      <td>Expected Results
      <td>At the end of this notebook, you should be able to: 
        <ul>
        <li>outcome one
        <li>outcome two
        </ul>
  </table>
</details>

In [ ]:
# library imports
import fsspec #testing
import hvplot.xarray #testing
import intake #testing
import os #testing
import warnings #testing
import rioxarray #testing
import dask #testing
import metpy #testing
import calendar #testing
import sparse 
import cf_xarray
import pygeohydro

from shapely.geometry import Polygon #testing
from dask.distributed import LocalCluster, Client #testing
from fsspec.implementations.ftp import FTPFileSystem #testing
from holoviews.streams import PolyEdit, PolyDraw #testing
from geocube.api.core import make_geocube #testing

import xarray as xr #testing
import geopandas as gpd #testing
import pandas as pd #testing
import geoviews as gv #testing
import dask.dataframe as dd #testing
import numpy as np #testing

warnings.filterwarnings('ignore')

# ceres_drb = xr.open_dataset(fs_read.open('s3://rsignellbucket2/hytest/tutorials/conus404_model_evaluation/ceres_drb.nc'), chunks={})

# Update to helper function after repo consolidation
## **Start a Dask client using an appropriate Dask Cluster** 
This is an optional step, but can speed up data loading significantly, especially when accessing data from the cloud.

In [ ]:
def configure_cluster(machine):
    ''' Helper function to configure cluster
    '''
    if machine == 'denali':
        from dask.distributed import LocalCluster, Client
        cluster = LocalCluster(threads_per_worker=1)
        client = Client(cluster)
    
    elif machine == 'tallgrass':
        from dask.distributed import Client
        from dask_jobqueue import SLURMCluster
        cluster = SLURMCluster(queue='cpu', cores=1, interface='ib0',
                               job_extra=['--nodes=1', '--ntasks-per-node=1', '--cpus-per-task=1'],
                               memory='6GB')
        cluster.adapt(maximum_jobs=30)
        client = Client(cluster)
        
    elif machine == 'local':
        import os
        import warnings
        from dask.distributed import LocalCluster, Client
        warnings.warn("Running locally can result in costly data transfers!\n")
        n_cores = os.cpu_count() # set to match your machine
        cluster = LocalCluster(threads_per_worker=n_cores)
        client = Client(cluster)
        
    elif machine in ['esip-qhub-gateway-v0.4']:   
        import sys, os
        sys.path.append(os.path.join(os.environ['HOME'],'shared','users','lib'))
        import ebdpy as ebd
        aws_profile = 'esip-qhub'  
        ebd.set_credentials(profile=aws_profile)

        aws_region = 'us-west-2'
        endpoint = f's3.{aws_region}.amazonaws.com'
        ebd.set_credentials(profile=aws_profile, region=aws_region, endpoint=endpoint)
        worker_max = 30
        client,cluster = ebd.start_dask_cluster(profile=aws_profile, worker_max=worker_max, 
                                              region=aws_region, use_existing_cluster=True,
                                              adaptive_scaling=True, wait_for_cluster=False, 
                                              worker_profile='Medium Worker', propagate_env=True)
        
    return client, cluster

### Setup your cluster

#### QHub...
Uncomment single commented spaces (#) to run

In [ ]:
# set machine
machine = 'esip-qhub-gateway-v0.4'

# use configure cluster helper function to setup dask
client, cluster = configure_cluster(machine)

#### or HPC
Uncomment single commented spaces (#) to run

In [ ]:
## set machine
# machine = os.environ['SLURM_CLUSTER_NAME']

## use configure_cluster helper function to setup dask
# client, cluster = configure_cluster(machine)

Dataset outline:
1. Read in the prepared dataset
2. Read in the HUC6 boundaries and transform to same coordinate reference system as prepared dataset
3. Make a data mask with the HUC6 boundaries to calculate zonal statistics
4. Compute zonal statistics with data mask and prepared data

Once all calculations are done: 

5. Combine each reference with benchmark into single dataset
6. Export gridded data zonal statistics
<br>

**CONUS404 zonal statistics**

## **Compute zonal statistics for gridded datasets**

In the last tutorial, we prepared three gridded datasets: CONUS404 (benchmark), PRISM (reference), and CERES-EBAF (reference). The goal of this section is compute [zonal statistics](https://gisgeography.com/zonal-statistics/) for each HUC6 zone in the Delaware River Basin (DRB) by using the [conservative regridding method put forth by Ryan Abernathy](https://discourse.pangeo.io/t/conservative-region-aggregation-with-xarray-geopandas-and-sparse/2715) to regrid and perform an area-weight analysis.

Dataset outline:
<ol>
    <li>Read in the prepared dataset</li>
    <li>Compute bounding bands for latitude and longitude (if necessary) then use these to create polygons in area-preserving CRS</li>
    <li>Read in the HUC6 boundaries and transform to same coordinate reference system as prepared dataset</li>
    <li>Overlay the dataset polygons over the HUC6 boundaries and create spatial weights matrices</li>
    <li>Perform matrix multiplication between The prepared dataset and the spatial weights matrices</li>
    <li>Perform zonal statistics</li>
</ol>

The following two functions will be used for regridding each dataset. Review them for now and an explanation of what they do will be provided when they are applied.

In [ ]:
def bounds_to_poly(x_bounds, y_bounds):
    """Return a polygon based on the x (longitude) and y (longitude) bounding band DataArrays"""
    return Polygon([
        (x_bounds[0], y_bounds[0]),
        (x_bounds[0], y_bounds[1]),
        (x_bounds[1], y_bounds[1]),
        (x_bounds[1], y_bounds[0])
    ])

def apply_weights_matmul_sparse(weights, data):
    """Apply weights in a sparse matrices to data and regrid"""
    assert isinstance(weights, sparse.SparseArray)
    assert isinstance(data, np.ndarray)
    data = sparse.COO.from_numpy(data)
    data_shape = data.shape
    n, k = data_shape[0], data_shape[1] * data_shape[2]
    data = data.reshape((n, k))
    weights_shape = weights.shape
    k_, m = weights_shape[0] * weights_shape[1], weights_shape[2]
    assert k == k_
    weights_data = weights.reshape((k, m))

    regridded = sparse.matmul(data, weights_data)
    assert regridded.shape == (n, m)
    return regridded.todense()

    assert isinstance(weights, sparse.SparseArray)
    assert isinstance(data, np.ndarray)
    data = sparse.COO.from_numpy(data)
    data_shape = data.shape
    # k = nlat * nlon
    n, k = data_shape[0], data_shape[1] * data_shape[2]
    data = data.reshape((n, k))
    weights_shape = weights.shape
    k_, m = weights_shape[0] * weights_shape[1], weights_shape[2]
    assert k == k_
    weights_data = weights.reshape((k, m))

    regridded = sparse.matmul(data, weights_data)
    assert regridded.shape == (n, m)
    return regridded.todense()

And the following `fsspec.filesystem` will be using to read in each dataset from an [Open Storage Network](https://www.openstoragenetwork.org/) bucket, which is read only.

In [ ]:
fs_read = fsspec.filesystem('s3', anon=True, skip_instance_cache=True,
                            client_kwargs={'endpoint_url': 'https://renc.osn.xsede.org'})

x = "x"
y = "y"

Setup the geometries for the DRB

In [ ]:
# bring in HUC6 boundaries found in the DRB
drb_gdf = pygeohydro.WBD("huc6", outfields=["huc6", "name"]).byids("huc6", ["020401", "020402"])

# area preserving crs
crs_area = "ESRI:53034"

# set CRS to match c404_drb
drb_gdf = drb_gdf.to_crs(crs_area)

#visualize
drb_gdf.plot(edgecolor="orange", facecolor="purple", linewidth=2.5)

**CONUS404 zonal statistics**

In [ ]:
# c404_drb = xr.open_dataset(fs_read.open('s3://rsignellbucket2/hytest/tutorials/conus404_model_evaluation/c404_drb.nc'), chunks={})
# url to c404_drb
c404_drb_url = 's3://rsignellbucket2/hytest/tutorials/conus404_model_evaluation/c404_drb.nc'

# open dataset
c404_drb = xr.open_dataset(fs_read.open(c404_drb_url), decode_coords='all') #, chunks={}

# crs
c404_crs = c404_drb.rio.crs.to_proj4()

# c404_drb

In [ ]:
c404_drb.PREC_ACC_NC.hvplot(x="x", y="y", rasterize=True)

Create the grid of c404_drb using any of the variables

In [ ]:
# set vars
c404_var = "TK"

# drop unneeded variable and coordinates
c404_grid = c404_drb[[c404_var]].drop(['time', 'lon', 'lat', c404_var]).reset_coords().load() #load in Richs code
c404_grid

And create bounding bands then stack into points

In [ ]:
# add bounds
c404_grid = c404_grid.cf.add_bounds(x)
c404_grid = c404_grid.cf.add_bounds(y)

# stack
c404_points = c404_grid.stack(point=(y,x))
c404_points

Next, use the `xarray apply_ufunc` function to apply the `bounds_to_poly` function above to the _c404_points_ DataSet.

In [ ]:
c404_boxes = xr.apply_ufunc(
    bounds_to_poly,
    c404_points.x_bounds,
    c404_points.y_bounds,
    input_core_dims=[("bounds",),  ("bounds",)],
    output_dtypes=[np.dtype('O')],
    vectorize=True
)
c404_boxes

Create geodataframe from boxes

In [ ]:
c404_grid_df= gpd.GeoDataFrame(
    data={"geometry": c404_boxes.values, "y": c404_boxes[y], "x": c404_boxes[x]},
    index=c404_boxes.indexes["point"],
    crs=c404_crs
)
c404_grid_df

In [ ]:
c404_grid_df.plot(edgecolor="red", facecolor="white", linewidth=0.05)

Overlay the two grids

In [ ]:
# convert DRB to conus404 crs
c404_drb_gdf = drb_gdf.to_crs(c404_crs)

#perform overly
c404_overlay = c404_grid_df.overlay(c404_drb_gdf, keep_geom_type=True)
c404_overlay.head()

In [ ]:
# plot overlay for single HUC6
c404_overlay[c404_overlay.huc6 == "020402"].geometry.plot(edgecolor='k')

Grid cell fractions

In [ ]:
c404_grid_cell_fraction = c404_overlay.geometry.area.groupby(c404_overlay.huc6).transform(lambda x: x / x.sum())

Sparse DataArray

In [ ]:
c404_multi_index = c404_overlay.set_index([y, x, "huc6"]).index
c404_df_weights = pd.DataFrame({"weights": c404_grid_cell_fraction.values}, index=c404_multi_index)

c404_ds_weights = xr.Dataset(c404_df_weights)

c404_weights_sparse = c404_ds_weights.unstack(sparse=True, fill_value=0.).weights

Matrix multiplication across each DataArray

In [ ]:
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    c404_precip_regridded = xr.apply_ufunc(
        apply_weights_matmul_sparse,
        c404_weights_sparse,
        c404_drb["PREC_ACC_NC"],
        join="left",
        input_core_dims=[["y", "x", "huc6"], ["y", "x"]],
        output_core_dims=[["huc6"]],
        dask="parallelized",
        meta=[np.ndarray((0,))]
    )

    c404_rnet_regridded = xr.apply_ufunc(
        apply_weights_matmul_sparse,
        c404_weights_sparse,
        c404_drb["RNET"],
        join="left",
        input_core_dims=[["y", "x", "huc6"], ["y", "x"]],
        output_core_dims=[["huc6"]],
        dask="parallelized",
        meta=[np.ndarray((0,))]
    )

    c404_tk_regridded = xr.apply_ufunc(
        apply_weights_matmul_sparse,
        c404_weights_sparse,
        c404_drb["TK"],
        join="left",
        input_core_dims=[["y", "x", "huc6"], ["y", "x"]],
        output_core_dims=[["huc6"]],
        dask="parallelized",
        meta=[np.ndarray((0,))]
    )

Merge DataArrays into Dataset

In [ ]:
c404_regridded = xr.Dataset({"PREC_NC_ACC":c404_precip_regridded, "RNET":c404_rnet_regridded, "TK": c404_tk_regridded})
c404_regridded = c404_regridded.drop("crs")
c404_regridded.attrs = c404_drb.attrs
c404_regridded

Covert to DataFrame

In [ ]:
c404_df = c404_regridded.load().to_dataframe()
c404_df

**PRISM zonal statistics**

PRISM has two variables: TK and PREC_ACC_NC

In [ ]:
# url to prism_drb
prism_drb_url = 's3://nhgf-development/workspace/tutorial/CONUS404/prism_drb.nc'

# open dataset
prism_drb = xr.open_dataset(fs_read.open('s3://rsignellbucket2/hytest/tutorials/conus404_model_evaluation/prism_drb.nc'))

# Create the grid of c404_drb using any of the variables

# set vars
prism_var = "TK"

# drop unneeded variable and coordinates
prism_grid = prism_drb[[prism_var]].drop(['time', 'lon', 'lat', prism_var]).reset_coords().load()


# create bounding bands then stack into points
# add bounds
c404_grid = c404_grid.cf.add_bounds(x)
c404_grid = c404_grid.cf.add_bounds(y)

# stack
c404_points = c404_grid.stack(point=(y,x))

# apply bounds_to
c404_boxes = xr.apply_ufunc(
    bounds_to_poly,
    c404_points.x_bounds,
    c404_points.y_bounds,
    input_core_dims=[("bounds",),  ("bounds",)],
    output_dtypes=[np.dtype('O')],
    vectorize=True
)


Create geodataframe from boxes

In [ ]:
c404_grid_df= gpd.GeoDataFrame(
    data={"geometry": c404_boxes.values, "y": c404_boxes[y], "x": c404_boxes[x]},
    index=c404_boxes.indexes["point"],
    crs=c404_crs
)
c404_grid_df

In [ ]:
c404_grid_df.plot(edgecolor="red", facecolor="white", linewidth=0.05)

Overlay the two grids

In [ ]:
# convert DRB to conus404 crs
c404_drb_gdf = drb_gdf.to_crs(c404_crs)

#perform overly
c404_overlay = c404_grid_df.overlay(c404_drb_gdf, keep_geom_type=True)
c404_overlay.head()

In [ ]:
# plot overlay for single HUC6
c404_overlay[c404_overlay.huc6 == "020402"].geometry.plot(edgecolor='k')

Grid cell fractions

In [ ]:
c404_grid_cell_fraction = c404_overlay.geometry.area.groupby(c404_overlay.huc6).transform(lambda x: x / x.sum())

Sparse DataArray

In [ ]:
c404_multi_index = c404_overlay.set_index([y, x, "huc6"]).index
c404_df_weights = pd.DataFrame({"weights": c404_grid_cell_fraction.values}, index=c404_multi_index)

c404_ds_weights = xr.Dataset(c404_df_weights)

c404_weights_sparse = c404_ds_weights.unstack(sparse=True, fill_value=0.).weights

Matrix multiplication across each DataArray

In [ ]:
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    c404_precip_regridded = xr.apply_ufunc(
        apply_weights_matmul_sparse,
        c404_weights_sparse,
        c404_drb["PREC_ACC_NC"],
        join="left",
        input_core_dims=[["y", "x", "huc6"], ["y", "x"]],
        output_core_dims=[["huc6"]],
        dask="parallelized",
        meta=[np.ndarray((0,))]
    )

    c404_rnet_regridded = xr.apply_ufunc(
        apply_weights_matmul_sparse,
        c404_weights_sparse,
        c404_drb["RNET"],
        join="left",
        input_core_dims=[["y", "x", "huc6"], ["y", "x"]],
        output_core_dims=[["huc6"]],
        dask="parallelized",
        meta=[np.ndarray((0,))]
    )

    c404_tk_regridded = xr.apply_ufunc(
        apply_weights_matmul_sparse,
        c404_weights_sparse,
        c404_drb["TK"],
        join="left",
        input_core_dims=[["y", "x", "huc6"], ["y", "x"]],
        output_core_dims=[["huc6"]],
        dask="parallelized",
        meta=[np.ndarray((0,))]
    )

Merge DataArrays into Dataset

In [ ]:
c404_regridded = xr.Dataset({"PREC_NC_ACC":c404_precip_regridded, "RNET":c404_rnet_regridded, "TK": c404_tk_regridded})
c404_regridded = c404_regridded.drop("crs")
c404_regridded.attrs = c404_drb.attrs
c404_regridded

Covert to DataFrame

In [ ]:
c404_df = c404_regridded.load().to_dataframe()
c404_df

In [ ]:
# url to prism_drb
prism_drb_url = 's3://nhgf-development/workspace/tutorial/CONUS404/prism_drb.nc'

fs = fsspec.filesystem("s3", anon=False, requester_pays=True, skip_instance_cache=True)

# open dataset
prism_drb = xr.open_dataset(fs.open(prism_drb_url), decode_coords="all")

# create an output grid
prism_out_grid = make_geocube(
    vector_data = drb_gdf,
    measurements=["huc6"],
    like=prism_drb
)

# add datarrays to grid
prism_out_grid["TK"] = (prism_drb.TK.dims, prism_drb.TK.values,
                         prism_drb.TK.attrs, prism_drb.TK.encoding)

prism_out_grid["PREC_ACC_NC"] = (prism_drb.PREC_ACC_NC.dims, prism_drb.PREC_ACC_NC.values,
                         prism_drb.PREC_ACC_NC.attrs, prism_drb.PREC_ACC_NC.encoding)

# groupby
prism_grouped = prism_out_grid.drop_vars("spatial_ref").groupby(prism_out_grid.huc6)

# Calculate the mean variables
prism_grid_mean = prism_grouped.mean().rename({"TK": "prism_TK_mean", 
                                       "PREC_ACC_NC": "prism_PREC_ACC_NC_mean", "time":"time_index"})

#convert to a dataframe
prism_zonal_stats = prism_grid_mean.to_dataframe().drop("spatial_ref", axis=1)

# reste index and add time back
prism_zonal_stats = prism_zonal_stats.reset_index(drop=False)
prism_zonal_stats["time"] = prism_drb.coords["time"][prism_zonal_stats["time_index"].values]
prism_zonal_stats.drop("time_index", axis=1, inplace=True)
prism_zonal_stats["time"] = prism_zonal_stats["time"].astype(str).str[:-3]

# change huc6 to string and pad with zeros
prism_zonal_stats["huc6"] = prism_zonal_stats["huc6"].astype(int).astype(str).str.zfill(6) # pads with 0's to make all column values lenght == 0

prism_zonal_stats

Merge the PRISM and CONUS404 zonals stats together based on the HUC6 code and time

In [ ]:
prism_c404_zonal = prism_zonal_stats.merge(c404_zonal_stats, left_on=['huc6', 'time'], right_on=['huc6', 'time'])
prism_c404_zonal.head()

We don't need the CONUS404 RNET value so we'll drop that column before exporting the data

In [ ]:
prism_c404_zonal.drop("c404_RNET_mean", axis=1, inplace=True)

Export the data

In [ ]:
prism_c404_zonal.to_parquet("s3://nhgf-development/workspace/tutorial/CONUS404/prism_c404_zonal.parquet")

**CERES-EBAF zonal statistics**

CERES-EBAF has a single variable: RNET

In [ ]:
# url to ceres_drb
ceres_drb_url = 's3://nhgf-development/workspace/tutorial/CONUS404/ceres_drb.nc'

fs = fsspec.filesystem("s3", anon=False, requester_pays=True, skip_instance_cache=True)

# open dataset
ceres_drb = xr.open_dataset(fs.open(ceres_drb_url), decode_coords="all", chunks={"time":10})
    
# create an output grid
ceres_out_grid = make_geocube(
    vector_data = drb_gdf,
    measurements=["huc6"],
    like=ceres_drb
)

# add datarrays to grid
ceres_out_grid["RNET"] = (ceres_drb.RNET.dims, ceres_drb.RNET.values,
                         ceres_drb.RNET.attrs, ceres_drb.RNET.encoding)

# groupby
ceres_grouped = ceres_out_grid.drop_vars("spatial_ref").groupby(ceres_out_grid.huc6)

# Calculate the mean variables
ceres_grid_mean = ceres_grouped.mean().rename({"RNET": "ceres_RNET_mean", "time":"time_index"})

#convert to a dataframe
ceres_zonal_stats = ceres_grid_mean.to_dataframe().drop("spatial_ref", axis=1)

# reste index and add time back
ceres_zonal_stats = ceres_zonal_stats.reset_index(drop=False)
ceres_zonal_stats["time"] = ceres_drb.coords["time"][ceres_zonal_stats["time_index"].values]
ceres_zonal_stats.drop("time_index", axis=1, inplace=True)
ceres_zonal_stats["time"] = ceres_zonal_stats["time"].astype(str).str[:-3]

# change huc6 to string and pad with zeros
ceres_zonal_stats["huc6"] = ceres_zonal_stats["huc6"].astype(int).astype(str).str.zfill(6) # pads with 0's to make all column values lenght == 0

ceres_zonal_stats

Merge the CERES-EBAF and CONUS404 zonals stats together based on the HUC6 code and time

In [ ]:
ceres_c404_zonal = ceres_zonal_stats.merge(c404_zonal_stats, left_on=['huc6', 'time'], right_on=['huc6', 'time'])
ceres_c404_zonal.head()

We don't need the CONUS404 TK and PREC_ACC_NC values so we'll drop these columns before exporting the data

In [ ]:
ceres_c404_zonal.drop(["c404_TK_mean", "c404_PREC_ACC_NC_mean"], axis=1, inplace=True)

In [ ]:
ceres_c404_zonal

Export the data

In [ ]:
ceres_c404_zonal.to_parquet("s3://nhgf-development/workspace/tutorial/CONUS404/ceres_c404_zonal.parquet")

## **Extract gridded values to points**

The goal of this section is extract values from CONUS404 where they intersect with station data. This process is described in article about the ESRI tool [Extract Values to Points](https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/extract-values-to-points.htm). This tabular data will then be exported for use in the next notebook, **CONUS404 Analysis**.

Dataset outline:
1. Read in the prepared dataset
2. Extract data from overlapping pixel at same time step as point
<br>

**Climate Reference Network point extraction**

In [ ]:
fs = fsspec.filesystem("s3", anon=False, requester_pays=True, skip_instance_cache=True)

crn_drb_df = pd.read_parquet(fs.open("s3://nhgf-development/workspace/tutorial/CONUS404/crn_drb.parquet"))

# create geodataframe
crn_drb = gpd.GeoDataFrame(crn_drb_df, crs=4326,
                       geometry=gpd.points_from_xy(crn_drb_df.LONGITUDE, 
                                                         crn_drb_df.LATITUDE))

# modify date field
crn_drb["DATE"] = crn_drb["DATE"].astype(str).str[:-3]

crn_drb.rename({"DATE": "time",
                "TK": "crn_TK", 
                "RNET": "crn_RNET", 
                "PREC_ACC_NC": "crn_PREC_ACC_NC"},
                  axis=1, inplace=True)

crn_drb.head()

Get coordinates from crn_drb to index c404_drb by

In [ ]:
# isolate single row and transform to c404_drb crs
crn_coords_gdf = crn_drb.iloc[[0]].to_crs(c404_crs)

# extract lat/long values
crn_lat = crn_coords_gdf.iloc[0]["geometry"].y
crn_lon = crn_coords_gdf.iloc[0]["geometry"].x

# time
crn_time_min = crn_drb_df["time"].min()
crn_time_max = crn_drb_df["time"].max()
crn_time_min, crn_time_max

# subset c404_drb to lat/long using nearest
c404_crn_sub = c404_drb.sel(x=crn_lon, y=crn_lat, method="nearest")

# slice to time-steps of crn_drb
c404_crn_sub = c404_crn_sub.sel(time=slice(crn_time_min, crn_time_max))

c404_crn_sub

Convert subset to dataframe and reorganize columns

In [ ]:
c404_sub_crn_df = c404_crn_sub.to_dataframe().reset_index(drop=False)

# trim columns
c404_sub_crn_df = c404_sub_crn_df[["time", "TK", "RNET", "PREC_ACC_NC"]]

# rename columns
c404_sub_crn_df.rename({"TK": "c404_TK", 
                    "RNET": "c404_RNET", 
                    "PREC_ACC_NC": "c404_PREC_ACC_NC"},
                  axis=1, inplace=True)

# trim time
c404_sub_crn_df["time"] = c404_sub_crn_df["time"].astype(str).str[:-3]

c404_sub_crn_df

Combine CONUS404 subset with CRN data

In [ ]:
crn_c404_point = crn_drb.merge(c404_sub_crn_df, on="time").reset_index(drop=False)

# drop columns
crn_c404_point.drop(["index", "LATITUDE", "LONGITUDE", "ID", "geometry"], axis=1, inplace=True)

crn_c404_point.head()

Export dataset

In [ ]:
crn_c404_point.to_parquet("s3://nhgf-development/workspace/tutorial/CONUS404/crn_c404_point.parquet")

**Historical Climate Network (HCN) point extraction**

The HCN data is different than the CRN data as the HCN data comes from multiple stations whereas the CRN data was from a single station. This will involve using multiple sets of geographic coordinates to extract data from CONUS404.

In [ ]:
# read in the HCN dataset
fs = fsspec.filesystem("s3", anon=False, requester_pays=True, skip_instance_cache=True)

hcn_drb_df = pd.read_parquet(fs.open("s3://nhgf-development/workspace/tutorial/CONUS404/hcn_drb.parquet"))

#rename columns
hcn_drb_df.rename({"DATE": "time",
                "TK": "hcn_TK",  
                "PREC_ACC_NC": "hcn_PREC_ACC_NC"},
                  axis=1, inplace=True)

# change DATE field to 
hcn_drb_df["time"] = hcn_drb_df["time"].astype(str).str[:-3]

hcn_drb_df.head()

Get a DataFrame of the station IDs, lats, and longs to use for extract data

In [ ]:
hcn_stations = hcn_drb_df.copy().drop(["time", "hcn_TK", "hcn_PREC_ACC_NC"], axis=1)
hcn_stations["LONGITUDE"] = pd.to_numeric(hcn_stations["LONGITUDE"])
hcn_stations["LATITUDE"] = pd.to_numeric(hcn_stations["LATITUDE"])

hcn_stations = hcn_stations.groupby('ID').mean().reset_index(drop=False)
# hcn_stations

Create a GeoDataFrame to convert the lat and long to the coordinate system of CONUS404

In [ ]:
hcn_stations_gdf = gpd.GeoDataFrame(hcn_stations, crs=4326,
                       geometry=gpd.points_from_xy(hcn_stations.LONGITUDE, 
                                                         hcn_stations.LATITUDE))

# transform to c404_drb crs
hcn_stations_gdf = hcn_stations_gdf.to_crs(c404_crs)

# extract lat/long values
hcn_stations_gdf["y"] = hcn_stations_gdf["geometry"].y
hcn_stations_gdf["x"] = hcn_stations_gdf["geometry"].x

#drop lat/lon/geo
hcn_stations_df = hcn_stations_gdf.drop(["LATITUDE", "LONGITUDE", "geometry"], axis=1)

Subset c404_drb to time period of HCN

In [ ]:
# time min/max
hcn_time_min = hcn_drb_df["time"].min()
hcn_time_max = hcn_drb_df["time"].max()

# slice c404 to HCN time
c404_hcn_timesub = c404_drb.sel(time=slice(hcn_time_min, hcn_time_max))

Use Dataframe rows to extract data from c404_drb

In [ ]:
# list of extracted data
c404_hcn_subs = []

for index, data in hcn_stations_df.iterrows():
    c404_hcn_sub_step = c404_hcn_timesub.sel(x=data.x, y=data.y, method="nearest").to_dataframe()
    c404_hcn_sub_step["ID"] = data.ID
    c404_hcn_subs.append(c404_hcn_sub_step)

# concat list of extracted data into single Dataframe
c404_hcn_sub = pd.concat(c404_hcn_subs)

#reset index
c404_hcn_sub.reset_index(drop=False, inplace=True)

# drop columns
c404_hcn_sub.drop(["RNET", "lon", "lat", "y", "x", "crs"], axis=1, inplace=True)

# rename columns
c404_hcn_sub.rename({"TK":"c404_TK",
                    "PREC_ACC_NC": "c404_PREC_ACC_NC"},
                   axis=1, inplace=True)

# trim time
c404_hcn_sub["time"] = c404_hcn_sub["time"].astype(str).str[:-3]

# c404_hcn_sub

Merge CONUS404 observations to HCH observations using the station ID and time

In [ ]:
hcn_c404_point = hcn_drb_df.merge(c404_hcn_sub, left_on=["ID", "time"], right_on=["ID", "time"])

# drop columns
hcn_c404_point.drop(["LATITUDE", "LONGITUDE"], axis=1, inplace=True)

hcn_c404_point.head()

Export the dataset

In [ ]:
hcn_c404_point.to_parquet("s3://nhgf-development/workspace/tutorial/CONUS404/hcn_c404_point.parquet")

Check all of the files that have been created in the data preparation notebooks

In [ ]:
fs = fsspec.filesystem("s3", anon=False, requester_pays=True, skip_instance_cache=True)

fs.ls("s3://nhgf-development/workspace/tutorial/CONUS404")

Shut down the client and cluster

In [ ]:
client.close(); cluster.shutdown()

# Next: CONUS404 Analysis notebook

Now that we have moved through our data preparation and calculated zonal and point statistics, we can move on to analyzing the differences between CONUS404 and the reference data in the CONUS404 Analysis notebook.

In [ ]:
# # Last code cell of the notebook
# import watermark.watermark as watermark
# print(watermark(iversions=True, python=True, machine=True, globals_=globals()))